In [ ]:
import json, pathlib, pprint, textwrap

SRC = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Conjunction/dataset/original/conjunction_test.json")         # 업로드된 원본
DST = pathlib.Path("/Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Conjunction/dataset/finetuning_data/conjunction_test.json")
with SRC.open(encoding="utf-8") as f:
    raw = json.load(f)

print(f"{len(raw):,} samples loaded")
pprint.pprint(raw[0], depth=2)

478 samples loaded
{'id': 'nikluge-2025-일상_대화_연결-test-1',
 'input': {'question': '위의 화자1의 발화 이전에 화자2가 했을 발화로 가장 적절한 것을 고르시오.',
           '대상발화': '뭘 해서 먹어야 되나 사실 걱정스럽죠잉.',
           '발화1': '끼니를 챙기는 것도 걱정이네요.',
           '발화2': '나이를 먹는 것도 걱정이네요.',
           '발화3': '욕을 먹는 것도 걱정이네요.',
           '발화위치': '선행문'}}


In [12]:
def key_to_letter(out: str) -> str:
    """'발화1'|'발화2'|'발화3' → 'A'|'B'|'C'"""
    return {"발화1": "A", "발화2": "B", "발화3": "C"}[out]

SYS_PROMPT = "당신은 주어진 발화를 듣고 질문에 답을 하는 assistant입니다. 주어진 발화를 듣고 화자1과 화자2의 대화에 대한 질문에 답하시오."

In [13]:
processed = []

for sample in raw:
    inp = sample["input"]
    user_msg = textwrap.dedent(f"""\
        [발화]
        화자1: {inp['대상발화']}

        [질문]
        아래의 선지 중, {inp['question']}

        [선지]
        A. 화자2: {inp['발화1']}
        B. 화자2: {inp['발화2']}
        C. 화자2: {inp['발화3']}

        A, B, C 를 제외한 그 무엇도 출력하지 마시오. 
        Return your answer only — no additional text.""")
    
    processed.append({
        "id": sample["id"],
        "messages": [
            {"role": "system",    "content": SYS_PROMPT},
            {"role": "user",      "content": user_msg},
            #{"role": "assistant", "content": key_to_letter(sample["output"])}
        ],
        "output": "발화1 | 발화2 | 발화3 중 하나"
    })

print("example ⇒"); pprint.pprint(processed[0], depth=3)


example ⇒
{'id': 'nikluge-2025-일상_대화_연결-test-1',
 'messages': [{'content': '당신은 주어진 발화를 듣고 질문에 답을 하는 assistant입니다. 주어진 발화를 듣고 '
                          '화자1과 화자2의 대화에 대한 질문에 답하시오.',
               'role': 'system'},
              {'content': '[발화]\n'
                          '화자1: 뭘 해서 먹어야 되나 사실 걱정스럽죠잉.\n'
                          '\n'
                          '[질문]\n'
                          '아래의 선지 중, 위의 화자1의 발화 이전에 화자2가 했을 발화로 가장 적절한 것을 '
                          '고르시오.\n'
                          '\n'
                          '[선지]\n'
                          'A. 화자2: 끼니를 챙기는 것도 걱정이네요.\n'
                          'B. 화자2: 나이를 먹는 것도 걱정이네요.\n'
                          'C. 화자2: 욕을 먹는 것도 걱정이네요.\n'
                          '\n'
                          'A, B, C 를 제외한 그 무엇도 출력하지 마시오. \n'
                          'Return your answer only — no additional text.',
               'role': 'user'}],
 'output': '발화1 | 발화2 | 발화3 중 하나'}


In [14]:
with DST.open("w", encoding="utf-8") as f:
    json.dump(processed, f, ensure_ascii=False, indent=2)

print(f"✔️ Saved {len(processed):,} samples → {DST.resolve()}")

✔️ Saved 478 samples → /Users/taeyoonkwack/Documents/HCLT-KACL-2025/Korean_Dialouge_Conjunction/dataset/finetuning_data/test.json
